## Playground for utilities to load, parse, and transform bank files


In [186]:
import pandas as pd
# import numpy as np
import json

from decimal import Decimal # To represent monetary values  https://docs.python.org/3/library/decimal.html  

In [187]:
# Data types for GL data
type TransactionID = str   # Unique identifier for each transaction - TBD / maybe a hash of the transaction details?
type TransactionDate = str # ... to be determined how to handle this
type TransactionAmount = Decimal # 
type DebitCreditIndicator = str # 'D' or 'C'
type TransactionDescription = str
type AccountID = str
type CurrencyUnit = str
type BusinessPartner = str
type Origin = str  # The original bank or credit card account that the transaction came from

# Can possibly be simplified by using @dataclass decorator
# Data type of GL item
class GLItem:
    def __init__(self, transactionID: TransactionID, transactionDate: TransactionDate, transactionAmount: TransactionAmount, 
                 debitCreditIndicator: DebitCreditIndicator, transactionDescription: TransactionDescription, accountID: AccountID, 
                 currencyUnit: CurrencyUnit, businessPartner: BusinessPartner, origin: Origin):
        self.transactionID = transactionID
        self.transactionDate = transactionDate
        self.transactionAmount = transactionAmount
        self.debitCreditIndicator = debitCreditIndicator
        self.transactionDescription = transactionDescription
        self.accountID = accountID
        self.currencyUnit = currencyUnit
        self.businessPartner = businessPartner
        self.Origin = origin

    def __str__(self):
        return f'{self.transactionID}, {self.transactionDate}, {self.transactionAmount}, {self.debitCreditIndicator}, {self.transactionDescription}, {self.accountID}, {self.currencyUnit}, {self.businessPartner}, {self.Origin}'

    def __repr__(self):
        return f'{self.transactionID}, {self.transactionDate}, {self.transactionAmount}, {self.debitCreditIndicator}, {self.transactionDescription}, {self.accountID}, {self.currencyUnit}, {self.businessPartner}, {self.Origin}'

In [188]:
# Read Wells Fargo test file into data frame
wfc_ColumnNames = ['Date', 'Amount', 'Unknown', 'CheckNo', 'Description']
# wfc_ColumnDataTypes = {'Amount': np.float16, 'Unknown': str, 'CheckNo': np.int16, 'Description': str}
wfc_RawData = pd.read_csv('Bank-Files/WF-test.csv', index_col=None, parse_dates=True, 
                         names=wfc_ColumnNames)
wfc_RawData.head(5)

,Date,Amount,Unknown,CheckNo,Description
0,01/02/2025,-283.00,*,NaN,PITCAIRN HOMEOWN Payment XXXXX8552 Theo Zimmer...
1,01/02/2025,-1353.48,*,NaN,WF HOME MTG AUTO PAY 010125 XXXXXX9336 THEO A ...
2,01/02/2025,-559.74,*,NaN,Wageworks Bat6098782 BID564 PID 32656565 CLAUD...
3,01/02/2025,-500.00,*,NaN,APPLECARD GSBANK PAYMENT 123024 55683437 Theo ...
4,12/31/2024,0.10,*,NaN,INTEREST PAYMENT


In [189]:
# Read bank descriptor
with open('BankDescriptor-WFC.json', 'r') as file:
    wfc_bankDescriptor = json.load(file)

print(wfc_bankDescriptor)

{'BankAccountDescription': 'Wells Fargo Checking', 'BankAccountNumber': '*7846', 'BalanceSheetAccount': 'BS-WF-CHK', 'AccountDeterminationFile': 'AccountDetermination-WFC.json'}


In [190]:
# Read account determination (JSON file) into dictionary

with open(wfc_bankDescriptor['AccountDeterminationFile'], 'r') as file:
    wfc_accountDetermination = json.load(file)

print(wfc_accountDetermination)

{'PITCAIRN HOMEOWN Payment': {'Account': 'EXP-542T', 'BP': 'Pitcairn HOA'}, 'WF HOME MTG AUTO PAY': {'Account': 'EXP-MOR-1311A', 'BP': 'Wells Fargo'}, 'Wageworks': {'Account': 'EXP-HC-CBR', 'BP': 'Wageworks'}, 'APPLECARD GSBANK PAYMENT': {'Account': 'BC-TRAN-CCPAY', 'BP': 'Apple Card'}, 'INTEREST PAYMENT': {'Account': 'REV-INT', 'BP': 'Wells Fargo'}}


In [191]:
print(wfc_accountDetermination['PITCAIRN HOMEOWN Payment'])

{'Account': 'EXP-542T', 'BP': 'Pitcairn HOA'}


In [192]:

for key, value in wfc_accountDetermination.items():
    print(f"String: {key}, Account: {value["Account"]}")

String: PITCAIRN HOMEOWN Payment, Account: EXP-542T
String: WF HOME MTG AUTO PAY, Account: EXP-MOR-1311A
String: Wageworks, Account: EXP-HC-CBR
String: APPLECARD GSBANK PAYMENT, Account: BC-TRAN-CCPAY
String: INTEREST PAYMENT, Account: REV-INT


In [193]:

for index, row in wfc_RawData.head(5).iterrows():
    print(f"Index: {index}")
    print(row.Date, row.Amount, row.Description)

Index: 0
01/02/2025 -283.0 PITCAIRN HOMEOWN Payment XXXXX8552 Theo Zimmermann
Index: 1
01/02/2025 -1353.48 WF HOME MTG AUTO PAY 010125 XXXXXX9336 THEO A ZIMMERMANN
Index: 2
01/02/2025 -559.74 Wageworks Bat6098782 BID564 PID 32656565 CLAUDIA ALVARADO
Index: 3
01/02/2025 -500.0 APPLECARD GSBANK PAYMENT 123024 55683437 Theo Zimmermann
Index: 4
12/31/2024 0.1 INTEREST PAYMENT


In [194]:
# Another way to loop the data frame
# Assuming df is your DataFrame
#def process_row(row):
#    print(row)

#wfc_RawData.apply(process_row, axis=1)

In [195]:
# Loop through the data frame and determine the account
for index, row in wfc_RawData.head(5).iterrows():
    print(f"Index: {index}")
    print(row.Date, row.Amount, row.Description)
    for key, value in wfc_accountDetermination.items():
        #print(f"String: {key}, Account: {value}")
        if key in row.Description:
            print(f"Account determination: String: {key}, Account: {value["Account"]}, Business Partner: {value["BP"]}")
            break
    # handle case when no account is found: ...
    

Index: 0
01/02/2025 -283.0 PITCAIRN HOMEOWN Payment XXXXX8552 Theo Zimmermann
Account determination: String: PITCAIRN HOMEOWN Payment, Account: EXP-542T, Business Partner: Pitcairn HOA
Index: 1
01/02/2025 -1353.48 WF HOME MTG AUTO PAY 010125 XXXXXX9336 THEO A ZIMMERMANN
Account determination: String: WF HOME MTG AUTO PAY, Account: EXP-MOR-1311A, Business Partner: Wells Fargo
Index: 2
01/02/2025 -559.74 Wageworks Bat6098782 BID564 PID 32656565 CLAUDIA ALVARADO
Account determination: String: Wageworks, Account: EXP-HC-CBR, Business Partner: Wageworks
Index: 3
01/02/2025 -500.0 APPLECARD GSBANK PAYMENT 123024 55683437 Theo Zimmermann
Account determination: String: APPLECARD GSBANK PAYMENT, Account: BC-TRAN-CCPAY, Business Partner: Apple Card
Index: 4
12/31/2024 0.1 INTEREST PAYMENT
Account determination: String: INTEREST PAYMENT, Account: REV-INT, Business Partner: Wells Fargo


In [196]:
class AccountDetermination:
    def __init__(self, json_file):
        with open(json_file, 'r') as file:
            self.account_determination = json.load(file)

    def determine_account(self, description: str):
        for substring, account in self.account_determination.items():
            if substring in description:
                return account
        return None

# Example usage:
# account_det = AccountDetermination('AccountDetermination-WFC.json')
# account = account_det.determine_account('INTEREST PAYMENT')
# print(account)

In [198]:
import hashlib
import sqlite3

#
account_determination_wfc = AccountDetermination('AccountDetermination-WFC.json')

for index, row in wfc_RawData.head(15).iterrows():
    print(f"Index: {index}", row.Date, row.Amount, row.Description, 
          f"Account: {account_determination_wfc.determine_account(row.Description)}")
    accounts = account_determination_wfc.determine_account(row.Description)
    if accounts is None:
        print(f"Account not found for description: {row.Description}")
        continue
    accountID = accounts['Account']
    businessPartner = accounts['BP']
    print(f"Account: {accountID}, Business Partner: {businessPartner}")
    transactionDate = row.Date
    transactionAmount: TransactionAmount = row.Amount
    print(f"Transaction Date: {transactionDate}, Transaction Amount: {transactionAmount}")
    transactionDescription = row.Description
    print(f"Transaction Description: {transactionDescription}")
    # set the currency unit
    currencyUnit = 'USD'
    # Set the debit or credit indicator (this is based on GL, no how banks handle it - just the opposite)
    # For a regular bank account, a deposit is a debit and a withdrawal is a credit to the bank balance sheet account
    if transactionAmount >= 0:
        debitCreditIndicator = 'D' # Debit to bank balance sheet account
    else:
        debitCreditIndicator = 'C' # Credit to bank balance sheet account
    # Set the origin
    Origin = wfc_bankDescriptor['BankAccountDescription']

    # set the transactionID
    # Create a hash of the complete row to use as transactionID
    row_data = f"{row.Date}{row.Amount}{row.Description}{Origin}"
    transactionID = hashlib.sha256(row_data.encode()).hexdigest()
    print(f"Transaction ID: {transactionID}")

    # Create a GL item
    gl_item = GLItem(transactionID, transactionDate, transactionAmount, debitCreditIndicator, transactionDescription, accountID, currencyUnit, businessPartner, Origin)
    print("--------------------")
    print(gl_item)
    print("--------------------")
    # write the GL item to a file
    #with open('GL-Files/gl_items.csv', 'a') as f:
    #    f.write(f'{gl_item.transactionID},{gl_item.transactionDate},{gl_item.transactionAmount},{gl_item.debitCreditIndicator},{gl_item.transactionDescription},{gl_item.accountID},{gl_item.currencyUnit},{gl_item.businessPartner},{gl_item.Origin}\n')

    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('GL-Files/gl_items.db')
    cursor = conn.cursor()

    # Create table if it doesn't exist
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS gl_items (
        transactionID TEXT PRIMARY KEY,
        transactionDate TEXT,
        transactionAmount REAL,
        debitCreditIndicator TEXT,
        transactionDescription TEXT,
        accountID TEXT,
        currencyUnit TEXT,
        businessPartner TEXT,
        origin TEXT
    )
    ''')

    # Check if the transactionID already exists in the database
    cursor.execute('SELECT transactionID FROM gl_items WHERE transactionID = ?', (transactionID,))
    if cursor.fetchone() is None:
        # Insert the GL item into the database
        cursor.execute('''
        INSERT INTO gl_items (transactionID, transactionDate, transactionAmount, debitCreditIndicator, transactionDescription, accountID, currencyUnit, businessPartner, origin)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (gl_item.transactionID, gl_item.transactionDate, gl_item.transactionAmount, gl_item.debitCreditIndicator, gl_item.transactionDescription, gl_item.accountID, gl_item.currencyUnit, gl_item.businessPartner, gl_item.Origin))
        conn.commit()
        print(f"Inserted GL item with transactionID: {transactionID}")
    else:
        print(f"GL item with transactionID: {transactionID} already exists in the database")

    # Close the database connection
    conn.close()

Index: 0 01/02/2025 -283.0 PITCAIRN HOMEOWN Payment XXXXX8552 Theo Zimmermann Account: {'Account': 'EXP-542T', 'BP': 'Pitcairn HOA'}
Account: EXP-542T, Business Partner: Pitcairn HOA
Transaction Date: 01/02/2025, Transaction Amount: -283.0
Transaction Description: PITCAIRN HOMEOWN Payment XXXXX8552 Theo Zimmermann
Transaction ID: 61f28bce358234c35c1d07ae6092769d2f1b4876a114e00089518b1e46387e6a
--------------------
61f28bce358234c35c1d07ae6092769d2f1b4876a114e00089518b1e46387e6a, 01/02/2025, -283.0, C, PITCAIRN HOMEOWN Payment XXXXX8552 Theo Zimmermann, EXP-542T, USD, Pitcairn HOA, Wells Fargo Checking
--------------------
GL item with transactionID: 61f28bce358234c35c1d07ae6092769d2f1b4876a114e00089518b1e46387e6a already exists in the database
Index: 1 01/02/2025 -1353.48 WF HOME MTG AUTO PAY 010125 XXXXXX9336 THEO A ZIMMERMANN Account: {'Account': 'EXP-MOR-1311A', 'BP': 'Wells Fargo'}
Account: EXP-MOR-1311A, Business Partner: Wells Fargo
Transaction Date: 01/02/2025, Transaction Amoun